# Q2.1 Classic Machine Learning Methods

## Pre Task

In [2]:
import pandas as pd
import numpy as np
from project_1.config import PROCESSED_DATA_DIR, PROJ_ROOT

SEED = 42

2025-03-21 18:41:08.481 | INFO     | project_1.config:<module>:11 - PROJ_ROOT path is: /Users/francescobondi/Desktop/stuff/ETH/FS25/ML for Healthcare/project-1-ml4hc


In [4]:
# Load the data from Parquet files
sets_dict = {}
sets = ["a", "b", "c"]

for set_name in sets:
    directory = PROCESSED_DATA_DIR / f"set_{set_name}_final.parquet"
    temp_set = pd.read_parquet(directory)
    sets_dict[f"set_{set_name}"] = temp_set

# Assure the loading was correct
print(sets_dict["set_a"].shape)
sets_dict["set_a"].head()


(183416, 43)


,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,Urine,WBC,pH,MechVent,TroponinT,ALP,ALT,AST,Bilirubin,TroponinI
0,132539.0,2025-03-10 00:00:00,0.0,-0.950526,-0.23008,-0.596332,1.671639,-0.013487,-0.832594,-0.109176,...,11.571429,0.753623,1.125,0.0,1.923077,0.132075,-0.176471,0.450704,1.545455,0.285714
1,132539.0,2025-03-10 01:00:00,0.0,-0.950526,-0.23008,-0.596332,1.967793,0.172112,-0.608431,-0.109176,...,2.857143,-0.420290,0.125,0.0,-0.246154,0.000000,-0.294118,0.436620,0.000000,-0.126984
2,132539.0,2025-03-10 02:00:00,0.0,-0.950526,-0.23008,-0.596332,-1.734132,0.125712,0.848629,0.830987,...,-0.357143,-0.014493,-0.875,0.0,0.000000,0.773585,-0.205882,-0.380282,0.181818,-0.095238
3,132539.0,2025-03-10 03:00:00,0.0,-0.950526,-0.23008,-0.596332,1.523562,0.380910,-0.832594,-0.579257,...,0.642857,0.188406,-0.375,0.0,0.215385,-0.698113,-0.588235,1.126761,-0.181818,4.650794
4,132539.0,2025-03-10 04:00:00,0.0,-0.950526,-0.23008,-0.596332,0.487023,-0.964680,1.483758,-0.814297,...,-0.142857,-1.144928,1.000,0.0,2.738462,-0.490566,-0.558824,-0.225352,0.363636,0.904762


ICU TYpe da cavare

In [5]:

# Define file names
file_names = ["Outcomes-a.txt", "Outcomes-b.txt", "Outcomes-c.txt"]

# Directory path
base_path = PROJ_ROOT / "data" / "data_1" / "predicting-mortality-of-icu-patients-the-physionet-computing-in-cardiology-challenge-2012-1.0.0"

# Read files into DataFrames containing all variables
outcomes_a, outcomes_b, outcomes_c = [pd.read_csv(base_path / name) for name in file_names]

# Extract only the "RecordID" and "In-hospital_death" column into separate DataFrames
death_a, death_b, death_c = [df[["RecordID", "In-hospital_death"]] for df in [outcomes_a, outcomes_b, outcomes_c]]
death_a.head()

#CHECK for missing values in the outcome data
print(death_a.isnull().sum())
print(death_b.isnull().sum())
print(death_c.isnull().sum())



RecordID             0
In-hospital_death    0
dtype: int64
RecordID             0
In-hospital_death    0
dtype: int64
RecordID             0
In-hospital_death    0
dtype: int64


In [6]:
# Check if each "In-hospital_death" column contains only 0 and 1
for name, df in zip(["a", "b", "c"], [death_a, death_b, death_c]):
    print(f"\nValue counts for death_{name}:\n")
    print(df["In-hospital_death"].value_counts())
    print("\nContains only 0 and 1:", df["In-hospital_death"].isin([0, 1]).all())



Value counts for death_a:

In-hospital_death
0    3446
1     554
Name: count, dtype: int64

Contains only 0 and 1: True

Value counts for death_b:

In-hospital_death
0    3432
1     568
Name: count, dtype: int64

Contains only 0 and 1: True

Value counts for death_c:

In-hospital_death
0    3415
1     585
Name: count, dtype: int64

Contains only 0 and 1: True


So 1 means he died. We are going to fit a classifier to predict death. 

## Task 1 - Prepare Data Sets (train_set, val_set, test_set)

In [7]:
# Define the sets you want to modify
sets = ["set_a", "set_b", "set_c"]

# Delete the column ICUType from each set
for set_name in sets:
    sets_dict[set_name].drop(columns=["ICUType"], inplace=True)

# Check if the column was deleted by printing all the columns of set_a
sets_dict["set_a"].columns

Index(['RecordID', 'Time', 'Gender', 'Height', 'Weight', 'Age', 'Albumin',
       'Cholesterol', 'DiasABP', 'HCO3', 'HCT', 'HR', 'Mg', 'MAP', 'Na',
       'NIDiasABP', 'NIMAP', 'NISysABP', 'SysABP', 'PaCO2', 'PaO2',
       'Platelets', 'RespRate', 'Temp', 'BUN', 'Creatinine', 'FiO2', 'GCS',
       'Glucose', 'K', 'Lactate', 'SaO2', 'Urine', 'WBC', 'pH', 'MechVent',
       'TroponinT', 'ALP', 'ALT', 'AST', 'Bilirubin', 'TroponinI'],
      dtype='object')

In [8]:
# Define training, validation and testing sets
train_set = sets_dict["set_a"]
val_set = sets_dict["set_b"]
test_set = sets_dict["set_c"]


In [9]:

#change class of RecordID to int32
train_set["RecordID"] = train_set["RecordID"].astype("int32")
val_set["RecordID"] = val_set["RecordID"].astype("int32")
test_set["RecordID"] = test_set["RecordID"].astype("int32")


In [10]:
# Merge the training, validation and testing sets with the corresponding death DataFrames
train_set = train_set.merge(death_a, on="RecordID")
val_set = val_set.merge(death_b, on="RecordID") 
test_set = test_set.merge(death_c, on="RecordID")

# Check if the merge was successful by printing the first 5 rows of the training set
train_set.head()

,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,WBC,pH,MechVent,TroponinT,ALP,ALT,AST,Bilirubin,TroponinI,In-hospital_death
0,132539,2025-03-10 00:00:00,0.0,-0.950526,-0.23008,-0.596332,1.671639,-0.013487,-0.832594,-0.109176,...,0.753623,1.125,0.0,1.923077,0.132075,-0.176471,0.450704,1.545455,0.285714,0
1,132539,2025-03-10 01:00:00,0.0,-0.950526,-0.23008,-0.596332,1.967793,0.172112,-0.608431,-0.109176,...,-0.420290,0.125,0.0,-0.246154,0.000000,-0.294118,0.436620,0.000000,-0.126984,0
2,132539,2025-03-10 02:00:00,0.0,-0.950526,-0.23008,-0.596332,-1.734132,0.125712,0.848629,0.830987,...,-0.014493,-0.875,0.0,0.000000,0.773585,-0.205882,-0.380282,0.181818,-0.095238,0
3,132539,2025-03-10 03:00:00,0.0,-0.950526,-0.23008,-0.596332,1.523562,0.380910,-0.832594,-0.579257,...,0.188406,-0.375,0.0,0.215385,-0.698113,-0.588235,1.126761,-0.181818,4.650794,0
4,132539,2025-03-10 04:00:00,0.0,-0.950526,-0.23008,-0.596332,0.487023,-0.964680,1.483758,-0.814297,...,-1.144928,1.000,0.0,2.738462,-0.490566,-0.558824,-0.225352,0.363636,0.904762,0


## Task 2 - Perform Aggregation on the Sets

In [11]:
# Define aggregation rules
aggregation_rules = {
    "Age": "last",
    "Gender": "last",
    "Height": "last",
    "Albumin": "last",
    "ALP": "last",
    "ALT": "last",
    "AST": "last",
    "Bilirubin": "last",
    "BUN": "last",
    "Cholesterol": "last",
    "Creatinine": "last",
    "DiasABP": "mean",
    "FiO2": "mean",
    "GCS": "min",
    "Glucose": "mean",
    "HCO3": "last",
    "HCT": "last",
    "HR": "mean",
    "K": "last",
    "Lactate": "max",
    "Mg": "last",
    "MAP": "mean",
    "MechVent": "last",
    "Na": "last",
    "NIDiasABP": "mean",
    "NIMAP": "mean",
    "NISysABP": "mean",
    "PaCO2": "last",
    "PaO2": "mean",
    "pH": "last",
    "Platelets": "last",
    "RespRate": "mean",
    "SaO2": "mean",
    "SysABP": "mean",
    "Temp": "max",
    "TroponinI": "max",
    "TroponinT": "max",
    "Urine": "sum",
    "WBC": "last",
    "Weight": "last",
    "In-hospital_death": "max"  # If any 1 exists for a patient, return 1
}

# Perform aggregation
train_aggregated = train_set.groupby("RecordID").agg(aggregation_rules).reset_index()
val_aggregated = val_set.groupby("RecordID").agg(aggregation_rules).reset_index()
test_aggregated = test_set.groupby("RecordID").agg(aggregation_rules).reset_index()


# Display the processed dataset
train_aggregated.head()

,RecordID,Age,Gender,Height,Albumin,ALP,ALT,AST,Bilirubin,BUN,...,RespRate,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,In-hospital_death
0,132539,-0.596332,0.0,-0.950526,-1.289901,-0.716981,-0.411765,0.802817,-0.181818,-0.588235,...,-0.302039,-0.427083,-0.108444,1.501788,6.095238,34.892308,43.142857,-0.318841,-0.230080,0
1,132540,0.667051,1.0,0.617613,-1.289901,-0.716981,-0.411765,0.802817,-0.181818,0.176471,...,-0.019156,-0.632653,0.001751,1.123672,6.095238,34.892308,37.022619,0.246377,-0.232705,0
2,132541,-1.170597,0.0,-0.429614,-1.141824,0.471698,1.529412,1.718310,1.909091,-0.882353,...,-0.018740,-1.166667,-0.053937,2.510098,6.095238,34.892308,30.971429,-0.782609,-1.077244,0
3,132543,0.207639,1.0,1.157978,1.967793,0.471698,-0.323529,-0.380282,-0.454545,-0.470588,...,-0.722062,-0.408163,-0.152114,-0.010676,6.095238,34.892308,135.657143,-0.536232,0.143617,0
4,132545,1.356169,0.0,-1.252050,0.338946,-0.716981,-0.411765,0.802817,-0.181818,0.411765,...,-0.023980,-0.414894,-0.184575,0.997634,6.095238,34.892308,7.114286,-0.985507,-0.809442,0


# Model 1 - Logistic Regression

In [12]:
# Separate Predictors (X) and Target (y)
X_train = train_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_train = train_aggregated["In-hospital_death"]

X_val = val_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_val = val_aggregated["In-hospital_death"]

X_test = test_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_test = test_aggregated["In-hospital_death"]

# Visualize the shape of the datasets
print(X_train.shape, y_train.shape)


(4000, 40) (4000,)


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

# Separate features and target for each set.
# Adjust the column name "target" to your actual target column name.

# Create and train the Logistic Regression classifier.
clf = LogisticRegression(random_state=SEED, max_iter=1000)
clf.fit(X_train, y_train)

# Optionally, evaluate on the validation set.
y_valid_proba = clf.predict_proba(X_val)[:, 1]  # probability for the positive class
roc_auc_valid = roc_auc_score(y_val, y_valid_proba)
auprc_valid = average_precision_score(y_val, y_valid_proba)
print(f"Validation ROC AUC: {roc_auc_valid:.3f}, AUPRC: {auprc_valid:.3f}")

# Evaluate on the test set.
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.790, AUPRC: 0.419
Test ROC AUC: 0.770, AUPRC: 0.392


# Model 2 - Random Forest

In [14]:
# Use Random Forest to predict the target
from sklearn.ensemble import RandomForestClassifier

# Create and train the Random Forest classifier.
clf = RandomForestClassifier(random_state=SEED)
clf.fit(X_train, y_train)

# Optionally, evaluate on the validation set.
y_valid_proba = clf.predict_proba(X_val)[:, 1]  # probability for the positive class
roc_auc_valid = roc_auc_score(y_val, y_valid_proba)
auprc_valid = average_precision_score(y_val, y_valid_proba)
print(f"Validation ROC AUC: {roc_auc_valid:.3f}, AUPRC: {auprc_valid:.3f}")

# Evaluate on the test set.
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.791, AUPRC: 0.431
Test ROC AUC: 0.778, AUPRC: 0.410


## A little comment on the results:

AUC Score is pretty good. However, AUPRC is not that high. A problem that could arise is class imbalance. In fact, we have two classes, where the size of the 0 class is 6x the size of the 1 class. However, all three sets share the same problem, so it should not be a big problem. Another possible implementation could be using a dimensionality reduction technique, or some feature engineering.

# Q2.1 - Feature Engineering

We could use solutions such as *_Feature Lagging_*, *_Temporal Differences_* or *_Rolling Statistics_* to better capture the time-series trends.

## Attempt 1 - Feature Lagging

Start by selecting the most prominent features (the ones with highest correlation)

In [15]:
def compute_patientwise_avg_acf(df, feature, lag=1):
    """
    Compute the average lag-1 autocorrelation for a given feature across patients.
    """
    acf_values = []
    for rid, group in df.groupby("RecordID"):
        series = group[feature].dropna()
        if len(series) < lag + 1:
            continue
        acf_val = series.corr(series.shift(lag))
        if pd.notna(acf_val):
            acf_values.append(acf_val)
    if len(acf_values) > 0:
        return np.mean(acf_values)
    else:
        return None

#! Using train_set because it's before the aggregation
# Assume your training DataFrame is named train_set and includes "RecordID", "Time", "target", plus dynamic features.
# Define the candidate features: exclude static ones ("RecordID", "Time", "target").
candidate_features = [col for col in train_set.columns if col not in ["RecordID", "Time", "In-hospital_death", "Age", "Weight", "Height", "Gender"]]

# Set threshold for absolute autocorrelation (lag 1)
threshold = 0.5
selected_features = []
for feature in candidate_features:
    avg_acf = compute_patientwise_avg_acf(train_set, feature, lag=1)
    if avg_acf is not None and abs(avg_acf) >= threshold:
        selected_features.append(feature)
        #print(f"Selected {feature} with average lag-1 ACF = {avg_acf:.3f}")

print("Features selected for lag augmentation:", selected_features)

/Users/francescobondi/anaconda3/envs/TUM/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/francescobondi/anaconda3/envs/TUM/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/Users/francescobondi/anaconda3/envs/TUM/lib/python3.12/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/francescobondi/anaconda3/envs/TUM/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Users/francescobondi/anaconda3/envs/TUM/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


Features selected for lag augmentation: ['HCO3', 'HCT', 'HR', 'Mg', 'Na', 'Temp', 'BUN', 'FiO2', 'GCS', 'K']


Check the number of NaN values in the entire train_set (as example). The number is 0 because we have a filled data set.

In [16]:
# Print number of NaN values in train set
print(train_set.isnull().sum().sum())

0


Then add the lag columns for those selected features.
WARNING! This operation creates NaN values inside the DataFrame

In [17]:
def add_lag_features_for_selected(df, selected_features, lags=[1,2]):
    """
    For each feature in selected_features, add lag features computed patient-wise.
    """
    df_augmented = df.copy()
    for feature in selected_features:
        if feature not in df_augmented.columns:
            continue
        for lag in lags:
            lag_col = f"{feature}_lag{lag}"
            df_augmented[lag_col] = df_augmented.groupby("RecordID")[feature].shift(lag)
    return df_augmented

# Augment train, validation, and test sets with lag features using the selected_features list.
train_set_aug = add_lag_features_for_selected(train_set, selected_features, lags=[1,2])
valid_set_aug = add_lag_features_for_selected(val_set, selected_features, lags=[1,2])
test_set_aug  = add_lag_features_for_selected(test_set,  selected_features, lags=[1,2])

# Print shapes
print(train_set_aug.shape, valid_set_aug.shape, test_set_aug.shape)

"""train_set_clean = train_set_aug.dropna()
valid_set_clean = valid_set_aug.dropna()
test_set_clean  = test_set_aug.dropna()

print(f"Train set shape after dropping NaNs: {train_set_clean.shape}")
print(f"Validation set shape after dropping NaNs: {valid_set_clean.shape}")
print(f"Test set shape after dropping NaNs: {test_set_clean.shape}")"""
train_set_aug.head()

(183416, 63) (183495, 63) (183711, 63)


,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,Temp_lag1,Temp_lag2,BUN_lag1,BUN_lag2,FiO2_lag1,FiO2_lag2,GCS_lag1,GCS_lag2,K_lag1,K_lag2
0,132539,2025-03-10 00:00:00,0.0,-0.950526,-0.23008,-0.596332,1.671639,-0.013487,-0.832594,-0.109176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,132539,2025-03-10 01:00:00,0.0,-0.950526,-0.23008,-0.596332,1.967793,0.172112,-0.608431,-0.109176,...,-0.514830,NaN,-0.176471,NaN,2.5,NaN,0.166667,NaN,1.875,NaN
2,132539,2025-03-10 02:00:00,0.0,-0.950526,-0.23008,-0.596332,-1.734132,0.125712,0.848629,0.830987,...,-2.090314,-0.514830,-0.058824,-0.176471,1.5,2.5,0.166667,0.166667,0.125,1.875
3,132539,2025-03-10 03:00:00,0.0,-0.950526,-0.23008,-0.596332,1.523562,0.380910,-0.832594,-0.579257,...,-2.090314,-2.090314,0.117647,-0.058824,0.0,1.5,0.166667,0.166667,0.000,0.125
4,132539,2025-03-10 04:00:00,0.0,-0.950526,-0.23008,-0.596332,0.487023,-0.964680,1.483758,-0.814297,...,-2.090314,-2.090314,-0.588235,0.117647,0.0,0.0,0.166667,0.166667,0.625,0.000


+20 features in total. Now print again the number of NaN values => It's increased because we inserted NaN values in the first 2 rows for each patient (lag-2 and lag-1)

In [18]:
# Print the number of NaN values
print(train_set_aug.isnull().sum().sum(), valid_set_aug.isnull().sum().sum(), test_set_aug.isnull().sum().sum())

119970 119930 119980


## Perform Aggregation on the Augmented DataFrames

By using "last" for the newly inserted columns, the NaN values should disappear.

Check if there are patients which have their last row with a NaN value

In [25]:
# Print the entire row of patients (RecordID) that have NaN values in their last row (last time point)
# Get the last row for each patient (grouped by RecordID)
last_rows = train_set_aug.groupby("RecordID").tail(1)

# Filter to get only those rows with at least one NaN value
missing_last = last_rows[last_rows.isna().any(axis=1)]

# Print the full rows for those patients
print("Patients with NaN values in their last row:")
missing_last


Patients with NaN values in their last row:


,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,Temp_lag1,Temp_lag2,BUN_lag1,BUN_lag2,FiO2_lag1,FiO2_lag2,GCS_lag1,GCS_lag2,K_lag1,K_lag2
19632,133628,2025-03-10 05:00:00,1.0,0.447939,-0.515385,1.356169,0.042792,-0.013487,-2.700621,-0.579257,...,-0.51483,NaN,-0.176471,NaN,2.5,NaN,0.166667,NaN,1.875,NaN
107728,138477,2025-03-10 10:00:00,0.0,-0.704120,-0.141250,-0.194347,-0.105285,-1.196678,0.064058,0.360906,...,-0.51483,NaN,-0.176471,NaN,2.5,NaN,0.166667,NaN,1.875,NaN
117892,139060,2025-03-10 06:00:00,1.0,0.723525,0.775927,-1.285450,0.635100,-1.405476,0.213500,-0.344216,...,-0.51483,NaN,-0.176471,NaN,2.5,NaN,0.166667,NaN,1.875,NaN
144254,140501,2025-03-10 00:00:00,1.0,0.447939,0.351470,-1.917142,1.671639,-0.013487,-0.832594,-0.109176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151735,140936,2025-03-10 00:00:00,1.0,1.327653,0.074041,-0.883465,1.671639,-0.013487,-0.832594,-0.109176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157699,141264,2025-03-10 00:00:00,0.0,-1.526556,-0.800253,1.471022,1.671639,-0.013487,-0.832594,-0.109176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Here for example, patient 133628 has NaN values. Let's explore his/her data.

In [27]:
# Print all the rows for RecordID = 133628
train_set_aug[train_set_aug["RecordID"] == 133628]

,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,Temp_lag1,Temp_lag2,BUN_lag1,BUN_lag2,FiO2_lag1,FiO2_lag2,GCS_lag1,GCS_lag2,K_lag1,K_lag2
19631,133628,2025-03-10 00:00:00,1.0,0.447939,-0.515385,1.356169,1.671639,-0.013487,-0.832594,-0.109176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19632,133628,2025-03-10 05:00:00,1.0,0.447939,-0.515385,1.356169,0.042792,-0.013487,-2.700621,-0.579257,...,-0.51483,NaN,-0.176471,NaN,2.5,NaN,0.166667,NaN,1.875,NaN


The problem occurs when a patient has not enough data (less than 3 measurements). Since only 6 patients out of 4000 share this problem, start by dropping them.

In [28]:
extended_aggregation_rules = aggregation_rules.copy()
for col in train_set_aug.columns:
    if "lag" in col:
        extended_aggregation_rules[col] = "last"

print("Extended aggregation rules:")
print(extended_aggregation_rules)

# Now perform aggregation on the train, validation, and test sets.
train_aggregated = train_set_aug.groupby("RecordID").agg(extended_aggregation_rules).reset_index()
val_aggregated   = valid_set_aug.groupby("RecordID").agg(extended_aggregation_rules).reset_index()
test_aggregated  = test_set_aug.groupby("RecordID").agg(extended_aggregation_rules).reset_index()

# Display the processed dataset (for example, for the train set)
print(train_aggregated.shape)
train_aggregated.head()

Extended aggregation rules:
{'Age': 'last', 'Gender': 'last', 'Height': 'last', 'Albumin': 'last', 'ALP': 'last', 'ALT': 'last', 'AST': 'last', 'Bilirubin': 'last', 'BUN': 'last', 'Cholesterol': 'last', 'Creatinine': 'last', 'DiasABP': 'mean', 'FiO2': 'mean', 'GCS': 'min', 'Glucose': 'mean', 'HCO3': 'last', 'HCT': 'last', 'HR': 'mean', 'K': 'last', 'Lactate': 'max', 'Mg': 'last', 'MAP': 'mean', 'MechVent': 'last', 'Na': 'last', 'NIDiasABP': 'mean', 'NIMAP': 'mean', 'NISysABP': 'mean', 'PaCO2': 'last', 'PaO2': 'mean', 'pH': 'last', 'Platelets': 'last', 'RespRate': 'mean', 'SaO2': 'mean', 'SysABP': 'mean', 'Temp': 'max', 'TroponinI': 'max', 'TroponinT': 'max', 'Urine': 'sum', 'WBC': 'last', 'Weight': 'last', 'In-hospital_death': 'max', 'HCO3_lag1': 'last', 'HCO3_lag2': 'last', 'HCT_lag1': 'last', 'HCT_lag2': 'last', 'HR_lag1': 'last', 'HR_lag2': 'last', 'Mg_lag1': 'last', 'Mg_lag2': 'last', 'Na_lag1': 'last', 'Na_lag2': 'last', 'Temp_lag1': 'last', 'Temp_lag2': 'last', 'BUN_lag1': 'last'

,RecordID,Age,Gender,Height,Albumin,ALP,ALT,AST,Bilirubin,BUN,...,Temp_lag1,Temp_lag2,BUN_lag1,BUN_lag2,FiO2_lag1,FiO2_lag2,GCS_lag1,GCS_lag2,K_lag1,K_lag2
0,132539,-0.596332,0.0,-0.950526,-1.289901,-0.716981,-0.411765,0.802817,-0.181818,-0.588235,...,0.745556,0.745556,-0.588235,-0.588235,0.0,-0.5,0.166667,0.166667,-0.125,-0.125
1,132540,0.667051,1.0,0.617613,-1.289901,-0.716981,-0.411765,0.802817,-0.181818,0.176471,...,-0.136714,-0.136714,0.176471,0.176471,-0.5,-0.5,0.166667,0.166667,-0.750,-0.750
2,132541,-1.170597,0.0,-0.429614,-1.141824,0.471698,1.529412,1.718310,1.909091,-0.882353,...,0.241402,1.375750,-0.882353,-0.882353,-0.5,-0.5,-1.500000,-1.500000,-0.500,-0.500
3,132543,0.207639,1.0,1.157978,1.967793,0.471698,-0.323529,-0.380282,-0.454545,-0.470588,...,-1.775217,-1.775217,-0.470588,-0.470588,0.0,-0.5,0.166667,0.166667,-0.375,-0.375
4,132545,1.356169,0.0,-1.252050,0.338946,-0.716981,-0.411765,0.802817,-0.181818,0.411765,...,-0.388792,-0.388792,0.411765,0.411765,0.0,-0.5,0.166667,0.166667,0.000,0.000


In [29]:
# Remove rows that include NaN values
train_aggregated_clean = train_aggregated.dropna()
val_aggregated_clean = val_aggregated.dropna()
test_aggregated_clean = test_aggregated.dropna()
train_aggregated_clean.shape, val_aggregated_clean.shape, test_aggregated_clean.shape


((3994, 62), (3992, 62), (3995, 62))

As we can see, we lost 6 patients in the train, 8 in the validation and only 5 in the test. I guess it's pretty reasonable.

### Evaluation with Logistic Regression

In [31]:
def prepare_xy(df):
    X = df.drop(columns=["In-hospital_death", "RecordID"])  # drop non-feature columns
    y = df["In-hospital_death"]  # target column
    return X, y

X_train, y_train = prepare_xy(train_aggregated_clean)
X_valid, y_valid = prepare_xy(val_aggregated_clean)
X_test,  y_test  = prepare_xy(test_aggregated_clean)

# Initialize and train Logistic Regression.
clf = LogisticRegression(random_state=SEED, max_iter=1000)
clf.fit(X_train, y_train)

# Evaluate on validation set.
y_valid_proba = clf.predict_proba(X_valid)[:, 1]
roc_auc_valid = roc_auc_score(y_valid, y_valid_proba)
auprc_valid = average_precision_score(y_valid, y_valid_proba)
print(f"Validation ROC AUC: {roc_auc_valid:.3f}, AUPRC: {auprc_valid:.3f}")

# Evaluate on test set.
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.840, AUPRC: 0.492
Test ROC AUC: 0.837, AUPRC: 0.491


### Evaluation with Random Forest

In [32]:
# Initialize and train Random Forest.
clf = RandomForestClassifier(random_state=SEED)
clf.fit(X_train, y_train)

# Evaluate on validation set.
y_valid_proba = clf.predict_proba(X_valid)[:, 1]
roc_auc_valid = roc_auc_score(y_valid, y_valid_proba)
auprc_valid = average_precision_score(y_valid, y_valid_proba)
print(f"Validation ROC AUC: {roc_auc_valid:.3f}, AUPRC: {auprc_valid:.3f}")

# Evaluate on test set.
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.837, AUPRC: 0.487
Test ROC AUC: 0.831, AUPRC: 0.497
